In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import os
import re
import nltk
import tensorflow
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
nltk.download('stopwords')
nltk.download('punkt')


train_df = pd.read_csv('/content/drive/MyDrive/346 project/training_set_rel3.tsv', sep='\t', encoding='ISO-8859-1', usecols=['essay_id', 'essay_set', 'essay', 'domain1_score']).dropna(axis=1)


stemmer = PorterStemmer()
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s.,\']', '', text)
    tokens = word_tokenize(text)
    stopwords_set = set(stopwords.words('english'))
    tokens = [stemmer.stem(word) for word in tokens if word not in stopwords_set]
    return ' '.join(tokens)

def preprocess_text2(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s.,\']', '', text)
    tokens = word_tokenize(text)
    stopwords_set = set(stopwords.words('english'))
    tokens = [stemmer.stem(word) for word in tokens if word not in stopwords_set]
    return tokens

train_df["essay_prepro"] = train_df["essay"].apply(preprocess_text)
X = train_df.drop(["essay_id", "essay_set", "essay", "domain1_score"], axis=1)
y = train_df["domain1_score"]
tokenized_documents = [preprocess_text2(doc) for doc in X["essay_prepro"]]

ukuran_vektor = 100
word2vec_model = Word2Vec(sentences=tokenized_documents, min_count=1, vector_size=ukuran_vektor, sg=1)
all_words = word2vec_model.wv.index_to_key

y = np.asarray(y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=92)

def document_vector(word2vec_model, doc_tokens):
    doc_vector = np.zeros(word2vec_model.vector_size)
    num_words = 0
    for word in doc_tokens:
        try:
            doc_vector += word2vec_model.wv[word]
            num_words += 1
        except KeyError:
            continue
    if num_words != 0:
        doc_vector /= num_words
    return doc_vector

X_train_vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_train["essay_prepro"]])
X_val_vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_val["essay_prepro"]])

encoder = OneHotEncoder()
essay_set_train_encoded = encoder.fit_transform(train_df.loc[X_train.index, 'essay_set'].values.reshape(-1, 1)).toarray()
essay_set_val_encoded = encoder.transform(train_df.loc[X_val.index, 'essay_set'].values.reshape(-1, 1)).toarray()

X_train_combined = np.concatenate((X_train_vec, essay_set_train_encoded), axis=1)
X_val_combined = np.concatenate((X_val_vec, essay_set_val_encoded), axis=1)

model = Sequential()
model.add(Input(shape=(ukuran_vektor+8,)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(61, activation='softmax'))

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
history = model.fit(X_train_combined, y_train, epochs=500, batch_size=256, validation_data=(X_val_combined, y_val), callbacks=[early_stopping])

loss, accuracy = model.evaluate(X_val_combined, y_val, verbose=0)
print(f'Accuracy: {accuracy*100:.2f}%')

# Plot loss
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot accuracy
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
plt.plot(accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Epoch 1/500
39/39 [==============================] - 1s 8ms/step - loss: 3.9794 - accuracy: 0.0806 - val_loss: 3.8781 - val_accuracy: 0.2395
Epoch 2/500
39/39 [==============================] - 0s 5ms/step - loss: 3.7908 - accuracy: 0.1874 - val_loss: 3.6462 - val_accuracy: 0.2303
Epoch 3/500
39/39 [==============================] - 0s 5ms/step - loss: 3.5227 - accuracy: 0.2152 - val_loss: 3.3183 - val_accuracy: 0.2330
Epoch 4/500
39/39 [==============================] - 0s 5ms/step - loss: 3.1787 - accuracy: 0.2283 - val_loss: 2.9354 - val_accuracy: 0.2552
Epoch 5/500
39/39 [==============================] - 0s 4ms/step - loss: 2.8775 - accuracy: 0.2292 - val_loss: 2.6900 - val_accuracy: 0.2676
Epoch 6/500
39/39 [==============================] - 0s 4ms/step - loss: 2.7278 - accuracy: 0.2244 - val_loss: 2.5670 - val_accuracy: 0.2685
Epoch 7/500
39/39 [==============================] - 0s 5ms/step - loss: 2.6350 - accuracy: 0.2307 - val_loss: 2.4839 - val_accuracy: 0.2688
Epoch 8/500
3